In [46]:
## ANN Regression using CHurn modelling 
# Predict the salary by training the model with other features available

import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
import pickle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [7]:
## load the csv file
data=pd.read_csv("churn_modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
# Delete the unwanted columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
## Gender Label encoding
gender_label_encoder=LabelEncoder()
data['Gender']=gender_label_encoder.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
## One hot Encoding for the Geography
geo_one_encode=OneHotEncoder()
geo_one_encoded=geo_one_encode.fit_transform(data[['Geography']])
geo_one_encoded.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [ ]:
## convert above information into dataframe with categorical column names

geo_one_encoded=pd.DataFrame(geo_one_encoded.toarray(),columns=geo_one_encode.get_feature_names_out())
geo_one_encoded.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [21]:
# Concat the whole dataframe
data=pd.concat([data.drop(['Geography'],axis=1),geo_one_encoded],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [26]:
## Let's split the data 
x=data.drop(['EstimatedSalary'],axis=1)
y=data['EstimatedSalary']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
## scale the data
standard_scaler=StandardScaler()
x_train_scaled=standard_scaler.fit_transform(x_train)


In [37]:
##Create a model for ANN Regression

model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)  # It is 1 because in Dense if you don't mention any activation function
              #then the default activation function is linear activation function whcih ic used for regression

])

C:\Users\nellu\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [38]:
# check model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
## setup model parameters
import tensorflow
optimizers= tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss= tensorflow.keras.losses.MeanAbsoluteError()

In [ ]:
## we need to save the pickle files
with open('Gender_label_encoder.pkl','wb') as file:
    pickle.dump(gender_label_encoder,file)
with open('Geography_one_hot.pkl','wb') as file:
    pickle.dump(geo_one_encode,file)
with open('Standard_scaler.pkl','wb') as file:
    pickle.dump(standard_scaler,file)


In [41]:
# Save the model
model.save('regression_model.keras')


In [50]:
## setup tensorboard
log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callbacks=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [55]:
# compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [56]:
# set up the EarlyStopping
early_stopping_callback=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)


In [57]:
# fit the data in model
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),
    epochs=100,callbacks=[early_stopping_callback,tensorboard_callbacks]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 66975.3438 - mae: 66975.3438 - val_loss: 63998.0703 - val_mae: 63998.0703
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 62144.3555 - mae: 62144.3555 - val_loss: 57750.8320 - val_mae: 57750.8320
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 55237.8125 - mae: 55237.8125 - val_loss: 52002.8164 - val_mae: 52002.8164
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 51891.8984 - mae: 51891.8984 - val_loss: 50665.8984 - val_mae: 50665.8984
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 51487.6445 - mae: 51487.6445 - val_loss: 50477.5625 - val_mae: 50477.5625
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 51345.7266 - mae: 51345.7266 - val_loss: 50395.7148 - val_mae: 50395.7148
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 51169.5703 - mae: 51169.5703 - val_loss: 50359.9219 - val_mae: 50359.9219
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - 

In [58]:
#launch the tensorboard
%load_ext tensorboard

In [60]:
#tensorboard 
%tensorboard --logdir logs/fit20251210-175546

Reusing TensorBoard on port 6006 (pid 24012), started 0:00:41 ago. (Use '!kill 24012' to kill it.)